In [100]:
import jax.numpy as jnp
import jax
import flax.linen as nn
import orbax.checkpoint as ocp
from jax import make_jaxpr

In [370]:
def conv3x3(in_channels, out_channels, stride = 1, padding = 1, bias = True):
    return nn.Conv(features= out_channels,
                   kernel_size= (3, 3), 
                   strides= (stride, stride),
                   padding= padding,
                   use_bias= bias)

In [375]:
def testit(model, input, key, make = False):
    if not isinstance(input, tuple): # handles objects with different no. of inputs UpConv and DownConv
        x = (input,)
    else:
        x = input
    key, split_key = jax.random.split(key)
    variables = model.init(split_key, *x)
    results = model.apply(variables, *x)
    print(f"\nVariable Shapes : \n{jax.tree.map(lambda x: x.shape, variables)} \n\nResults Shapes: {jax.tree.map(lambda x : x.shape, results)}")
    if make:
        print(f"\n\n\nResults : {results}")
        return make_jaxpr(lambda variables, x: model.apply(variables, x))(variables, x)
    return

In [188]:
key = jax.random.key(0)

In [372]:
key, split_key = jax.random.split(key)
testit(conv3x3(3, 3,), jnp.ones((1, 3, 3, 1)), key)


Variable Shapes : 
{'params': {'bias': (3,), 'kernel': (3, 3, 1, 3)}} 

Results Shapes: (1, 3, 3, 3)


In [367]:
def upconv2x2(in_channels, out_channels, mode='transpose'):
    '''
    Upsample not implemented, need to find alternative
    '''
    return nn.ConvTranspose(features= out_channels,
                            kernel_size= (2, 2),
                            strides= (2, 2))

In [369]:
key, split_key = jax.random.split(key)
testit(upconv2x2(3, 3,), jnp.ones((1, 3, 3, 1)), key)


Variable Shapes : 
{'params': {'bias': (3,), 'kernel': (2, 2, 1, 3)}} 

Results Shapes: (1, 6, 6, 3)


In [195]:
def conv1x1(in_channels, out_channels):
    return nn.Conv(features= out_channels, 
                   kernel_size= 1,
                   strides= 1)

In [293]:
key, split_key = jax.random.split(key)
testit(conv1x1(3, 3,), jnp.ones((1, 3)), key)


Variable Shapes : 
{'params': {'bias': (3,), 'kernel': (1, 3, 3)}} 

Results Shapes: (1, 3)


In [396]:
class DownConv(nn.Module):
    in_channels : int
    out_channels : int
    pooling : bool

    def setup(self):
        self.conv1 = conv3x3(self.in_channels, self.out_channels)
        self.conv2 = conv3x3(self.out_channels, self.out_channels)
        self.conv3 = conv3x3(self.out_channels, self.out_channels)

    def __call__(self, x):
        
        xskip = self.conv1(x)
        x = nn.relu(self.conv2(xskip))
        x = nn.relu(self.conv3(x) + xskip)
        before_pool = x

        if self.pooling:
            x = nn.max_pool(x,
                            window_shape= (2, 2),
                            strides= (2, 2))
            
        return x, before_pool


In [414]:
key, split_key = jax.random.split(key)
testit(DownConv(3, 3, True), jnp.ones((1, 3, 3, 3)), key = key, make = False)


Variable Shapes : 
{'params': {'conv1': {'bias': (3,), 'kernel': (3, 3, 3, 3)}, 'conv2': {'bias': (3,), 'kernel': (3, 3, 3, 3)}, 'conv3': {'bias': (3,), 'kernel': (3, 3, 3, 3)}}} 

Results Shapes: ((1, 1, 1, 3), (1, 3, 3, 3))


In [389]:
class UpConv(nn.Module):
    in_channels : int
    out_channels : int
    merge_mode : str = 'concat'
    up_mode : str = 'transpose'

    def setup(self):
        self.upconv = upconv2x2(self.in_channels, self.out_channels,mode=self.up_mode)
        self.conv1 = conv3x3(self.out_channels, self.out_channels) ## refine for flax
        self.conv2 = conv3x3(self.out_channels, self.out_channels)
        self.conv3 = conv3x3(self.out_channels, self.out_channels)

    def __call__(self, from_down, from_up):
        
        from_up = self.upconv(from_up) 
        if self.merge_mode == 'concat':
            x = jnp.concatenate([from_up, from_down], axis = -1) # check axis channel is last for jax
        else:
            x = from_up + from_down

        xskip = self.conv1(x) 
        x = nn.relu(self.conv2(xskip))
        x = nn.relu(self.conv3(x) + xskip)

        return x


In [417]:
key, split_key = jax.random.split(key)
a = jnp.ones((1, 4, 4, 64))
b = jnp.ones((1, 2, 2, 64))
x = (a, b)
testit(UpConv(3, 128), input = x, key = key, make = False)


Variable Shapes : 
{'params': {'conv1': {'bias': (128,), 'kernel': (3, 3, 192, 128)}, 'conv2': {'bias': (128,), 'kernel': (3, 3, 128, 128)}, 'conv3': {'bias': (128,), 'kernel': (3, 3, 128, 128)}, 'upconv': {'bias': (128,), 'kernel': (2, 2, 64, 128)}}} 

Results Shapes: (1, 4, 4, 128)


In [377]:
import torch
print(torch.sin(torch.tensor([5])))
print(jnp.sin(5))

tensor([-0.9589])
-0.9589243


In [405]:
class UN(nn.Module):
    levels : int
    channels : int = 3
    depth : int = 5
    start_filts : int = 64
    up_mode : str = 'transpose'
    merge_mode : str = 'add'

    def setup(self):
        if self.up_mode not in ('transpose', 'upsample'):
            raise ValueError("\"{}\" is not a valid mode for "
                             "upsampling. Only \"transpose\" and "
                             "\"upsample\" are allowed.".format(self.up_mode))
        
        if self.merge_mode not in ('concat', 'add'):
            raise ValueError("\"{}\" is not a valid mode for"
                             "merging up and down paths. "
                             "Only \"concat\" and "
                             "\"add\" are allowed.".format(self.up_mode))

        if self.up_mode == 'upsample' and self.merge_mode == 'add':
            raise ValueError("up_mode \"upsample\" is incompatible "
                             "with merge_mode \"add\" at the moment "
                             "because it doesn't make sense to use "
                             "nearest neighbour to reduce "
                             "depth channels (by half).")
        
        down_convs = []
        up_convs = []

        for i in range(self.depth):
            ins = self.channels * self.levels if i == 0 else outs
            outs = self.start_filts*(2**i)
            pooling = True if i < self.depth-1 else False

            module = DownConv(ins, outs, pooling=pooling)
            down_convs.append(module)
        self.down_convs = down_convs

        for i in range(self.depth-1):
            ins = outs
            outs = ins // 2
            module = UpConv(ins, outs, up_mode=self.up_mode,merge_mode=self.merge_mode)
            up_convs.append(module)
        self.up_convs = up_convs

        self.conv_final = conv1x1(outs, self.channels)



    def __call__(self, x):
        
        stack = None
        factor = 10.0
        for i in range (self.levels):
            scale = x.copy()*(factor**(-i))
            scale = jnp.sin(scale)
            if stack is None:
                stack = scale
            else:
                stack = jnp.concatenate([stack,scale],axis = -1)
        
        x = stack
        
        encoder_outs = []
        for i, module in enumerate(self.down_convs):
            x, before_pool = module(x)
            encoder_outs.append(before_pool)

        for i, module in enumerate(self.up_convs):
            before_pool = encoder_outs[-(i+2)]
            x = module(before_pool, x)
        
        x = self.conv_final(x)

        return x

In [415]:
key, split_key = jax.random.split(key)
a = jnp.ones((1, 128, 128, 3))
# b = jnp.ones((1, 3, 3, 64))
x = a
testit(UN(levels= 10), input = x, key = key, make = False)

2024-07-15 05:35:32.337598: W external/xla/xla/tsl/framework/bfc_allocator.cc:291] Allocator (GPU_0_bfc) ran out of memory trying to allocate 4.27GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.



Variable Shapes : 
{'params': {'conv_final': {'bias': (3,), 'kernel': (1, 64, 3)}, 'down_convs_0': {'conv1': {'bias': (64,), 'kernel': (3, 3, 30, 64)}, 'conv2': {'bias': (64,), 'kernel': (3, 3, 64, 64)}, 'conv3': {'bias': (64,), 'kernel': (3, 3, 64, 64)}}, 'down_convs_1': {'conv1': {'bias': (128,), 'kernel': (3, 3, 64, 128)}, 'conv2': {'bias': (128,), 'kernel': (3, 3, 128, 128)}, 'conv3': {'bias': (128,), 'kernel': (3, 3, 128, 128)}}, 'down_convs_2': {'conv1': {'bias': (256,), 'kernel': (3, 3, 128, 256)}, 'conv2': {'bias': (256,), 'kernel': (3, 3, 256, 256)}, 'conv3': {'bias': (256,), 'kernel': (3, 3, 256, 256)}}, 'down_convs_3': {'conv1': {'bias': (512,), 'kernel': (3, 3, 256, 512)}, 'conv2': {'bias': (512,), 'kernel': (3, 3, 512, 512)}, 'conv3': {'bias': (512,), 'kernel': (3, 3, 512, 512)}}, 'down_convs_4': {'conv1': {'bias': (1024,), 'kernel': (3, 3, 512, 1024)}, 'conv2': {'bias': (1024,), 'kernel': (3, 3, 1024, 1024)}, 'conv3': {'bias': (1024,), 'kernel': (3, 3, 1024, 1024)}}, 'up

In [ ]:
def photonLoss(result, target):
    expEnergy = jnp.exp(result)
    perImage = -jnp.mean(result*target, )

In [426]:
import numpy as np
x = np.arange(9) + 1.
x.reshape((1, 3, 3, 1))

array([[[[1.],
         [2.],
         [3.]],

        [[4.],
         [5.],
         [6.]],

        [[7.],
         [8.],
         [9.]]]])

In [427]:
dummy_result = torch.from_numpy(x)
dummy_target = torch.from_numpy(np.ones_like(x.reshape((1, 3, 3, 1))))

In [433]:
def photonLossTorch(result, target):
        expEnergy = torch.exp(result)
        return -torch.mean(result*target )

In [434]:
photonLossTorch(dummy_result, dummy_target)

tensor(-5., dtype=torch.float64)